## How are bills proposed/sponsored by Republicans in a Democratic-controlled legislature are just out-right ignored or whether they're strung along with endless amendments.

### Import Necessary Modules

In [1]:
import requests
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import re


# Turns out just because I want to use selenium doesn't make it make sense to do so for this context.

# Don't mind if there are two errors, those are due to either my connection quitting, or my keyboard interupting the program.

### Scraping all(most) of the bill data from leg.colorado.gov

In [ ]:



base_url = "https://leg.colorado.gov/bill-search?field_sessions=All&sort_bef_combine=field_bill_number%20ASC&field_chamber=All&field_bill_type=All&page="

# lists to store data
list_of_bills = []
bill_node_titles = []
bill_node_links = []
bill_desc = []
bill_last_action = []
bill_governor_signed = []
bill_sponsors_cleaned = []
bill_subjects = []
bill_session = []
bill_type = []


page_num = 0

while True:
    # Lets soupify page by page! It doesn't take as long as making get requests in ATLAS without an ethernet
    url = base_url + str(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
  
    if len(list_of_bills) > 6000: # selected this number to stop scraping.
        break 
    # create a base soup to then iterate through searching for the details!
    bill_containers = soup.find_all('div', class_='views-row')

    for container in bill_containers:
        bill_div = container.find('div', {'class': 'field-item even'})
        bill_text = bill_div.get_text(strip=True) if bill_div else None # if xxx else None is required for making array shape the correct size
        list_of_bills.append(bill_text) # it works as a data cleaning as it collects ensuring the array is the same size.
        # extract node title
        title_tag = container.find('h4', {'class': 'node__title node-title'})
        title = title_tag.get_text(strip=True) if title_tag else None
        link = title_tag.find('a')['href'] if title_tag and title_tag.find('a') else None
        bill_node_titles.append(title)
        bill_node_links.append(link)

        # extract description   
        desc_tag = container.find('div', {'class': 'field field-name-field-bill-long-title field-type-text-long field-label-hidden'})
        description = desc_tag.get_text(strip=True) if desc_tag else None
        bill_desc.append(description)

        action_tag = container.find('div', {'class': 'bill-last-action search-result-single-item'})
        last_action = action_tag.get_text(strip=True) if action_tag else None
        bill_last_action.append(last_action)
        governor_signed = 1 if last_action and 'Governor Signed' in last_action else 0
        bill_governor_signed.append(governor_signed)


        # extract sponsors
        sponsor_tag = container.find('div', {'class': 'bill-sponsors search-result-single-item'})
        sponsors = sponsor_tag.get_text(strip=True).replace("Sponsors:", "").replace("|", ", ").strip() if sponsor_tag else None
        bill_sponsors_cleaned.append(sponsors)

        # extract subject
        subject_tag = container.find('div', {'class': 'field field-name-field-subjects field-type-entityreference field-label-hidden'})
        subjects = subject_tag.get_text(strip=True) if subject_tag else None
        # add regex here or...? something to clean the lack of spaces between entries per bill
        bill_subjects.append(subjects)


        # extract session
        
        session_tag = container.find('div', {'class': 'field field-name-field-sessions field-type-entityreference field-label-hidden'})
        session = session_tag.get_text(strip=True) if desc_tag else None
        bill_session.append(session)

        # #bill-type search-result-single-item
        type_tag = container.find('div', {'class': 'bill-type search-result-single-item'})
        types = type_tag.get_text(strip=True) if desc_tag else None
        bill_type.append(types)

        if ConnectionError or KeyboardInterrupt: # bad attempt at error handling when the internet cuts out...
                df_bills = pd.DataFrame({
            'Bill Number': list_of_bills,
            'Title': bill_node_titles,
            'Link': bill_node_links,
            'Description': bill_desc,
            'Last Action': bill_last_action,
            'Governor Signed': bill_governor_signed,
            'Sponsors': bill_sponsors_cleaned,
            'Subjects':bill_subjects,
            'Session':bill_session,
            'Types':bill_type
        })

    #change the counter to load the next page
    page_num += 1


    # I love to watch my progress 
    print("Bill Numbers:", len(list_of_bills))
    print("Titles:", len(bill_node_titles))
    print("Links:", len(bill_node_links))
    print("Descriptions:", len(bill_desc))
    print("Last Actions:", len(bill_last_action))
    print("Governor Signed:", len(bill_governor_signed))
    print("Sponsors:", len(bill_subjects))
    print("Sponsors:", len(bill_sponsors_cleaned))
    print("Session:", len(bill_session))
    print("Bill Types:", len(bill_type))
    print("------------------------")

    df_bills = pd.DataFrame({
        'Bill Number': list_of_bills,
        'Title': bill_node_titles,
        'Link': bill_node_links,
        'Description': bill_desc,
        'Last Action': bill_last_action,
        'Governor Signed': bill_governor_signed,
        'Sponsors': bill_sponsors_cleaned,
        'Subjects':bill_subjects,
        'Session':bill_session,
        'Types':bill_type
    })

# clean up datetimes:

# cool tool I found for regex: https://www.regextranslator.com/
    
    df_bills['Last Action Date'] = df_bills['Last Action'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
    df_bills['Last Action Date'] = pd.to_datetime(df_bills['Last Action Date'], format='%m/%d/%Y')


print(df_bills)


In [ ]:

df_bills

### Run this cell to convert the dataframe to a csv

In [ ]:


#df_bills.to_csv("co-leg-bills.csv")

### Run this cell to read in said csv

In [2]:
df_bills = pd.read_csv('co-leg-bills.csv')
df_bills

,Unnamed: 0,Bill Number,Title,Link,Description,Last Action,Governor Signed,Sponsors,Subjects,Session,Types,Last Action Date
0,0,HB16-1001,State Contract Certify Compliance With Equal P...,/bills/hb16-1001,Concerning a requirement that a business perfo...,Last Action:03/30/2016 | Senate Committee on S...,0,"Rep. J. Danielson, Rep. J. Buckner, Sen. J. Ul...",Labor & EmploymentState Government,2016 Regular Session,Type:Bill,2016-03-30
1,1,HB16-1002,Employee Leave Attend Child's Academic Activities,/bills/hb16-1002,Concerning the ability of an employee of an em...,Last Action:03/9/2016 | Senate Committee on St...,0,"Rep. J. Buckner, Sen. A. Kerr",Education & School Finance (Pre & K-12)Labor &...,2016 Regular Session,Type:Bill,2016-03-09
2,2,HB16-1003,Middle Class College Savings Act,/bills/hb16-1003,Concerning the state income tax deductions for...,Last Action:05/5/2016 | Senate Committee on St...,0,"Rep. B. Pettersen, Rep. D. Young, Sen. M. Merr...",Fiscal Policy & TaxesHigher Education,2016 Regular Session,Type:Bill,2016-05-05
3,3,HB16-1004,Measurable Goals Deadlines Colorado Climate Ac...,/bills/hb16-1004,Concerning a requirement to include measurable...,Last Action:03/30/2016 | Senate Committee on A...,0,"Rep. F. Winter, Rep. J. Arndt, Sen. J. Kefalas",EnergyNatural Resources & Environment,2016 Regular Session,Type:Bill,2016-03-30
4,4,HB16-1005,Residential Precipitation Collection,/bills/hb16-1005,Concerning the use of rain barrels to collect ...,Last Action:05/12/2016 | Governor Signed,1,"Rep. D. Esgar, Rep. J. Danielson, Sen. M. Merr...",Water,2016 Regular Session,Type:Bill,2016-05-12
...,...,...,...,...,...,...,...,...,...,...,...,...
5320,5320,SB21-118,Alternative Response Mistreatment At-risk Adults,/bills/sb21-118,Concerning the creation of an alternative resp...,Last Action:06/17/2021 | Governor Signed,1,"Sen. J. Ginal, Sen. B. Gardner, Rep. R. Pelton...",Human Services,2021 Regular Session,Type:Bill,2021-06-17
5321,5321,SB21-119,Increasing Access To High-Quality Credentials,/bills/sb21-119,Concerning increasing access in high school to...,Last Action:06/30/2021 | Governor Signed,1,"Sen. J. Bridges, Sen. P. Lundeen, Rep. D. Esga...",Education & School Finance (Pre & K-12),2021 Regular Session,Type:Bill,2021-06-30
5322,5322,SB21-120,Open Caption Requirement For Movie Theaters,/bills/sb21-120,Concerning providing open captioning in movie ...,Last Action:03/10/2021 | Senate Committee on B...,0,"Sen. J. Ginal, Sen. J. Danielson, Rep. D. Orti...",Business & Economic DevelopmentHuman Services,2021 Regular Session,Type:Bill,2021-03-10
5323,5323,SB21-121,Revised Uniform Unclaimed Property Act,/bills/sb21-121,"Concerning modifications to the ""Revised Unifo...",Last Action:04/15/2021 | Governor Signed,1,"Sen. C. Hansen, Sen. K. Priola, Rep. R. Bocken...",State Government,2021 Regular Session,Type:Bill,2021-04-15


### Lets get more data based on bill links!

In [ ]:

base_url = "https://leg.colorado.gov"

bill_numbers = []
bill_titles = []
bill_summaries = []
bill_sessions = []
bill_sponsors = []


for link in df_bills['Link']:
        # page content
        response = requests.get(base_url + link)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # bill number
        bill_number = soup.find('div', class_='field-item even').get_text(strip=True)
        bill_numbers.append(bill_number)
        
        # bill title
        bill_title = soup.find('h1', class_='node-title').get_text(strip=True)
        bill_titles.append(bill_title)
        
        #bill summary
        bill_summary_div = soup.find('div', class_='field field-name-field-bill-summary field-type-text-long field-label-hidden')
        bill_summary = bill_summary_div.get_text(strip=True) if bill_summary_div else None
        bill_summaries.append(bill_summary)
        
        # session information
        session_div = soup.find('div', class_='bill-session')
        session = session_div.get_text(strip=True) if session_div else None
        bill_sessions.append(session)

        # sponsor information
        sponsor_div = soup.find('div', id='block-cga-bills-bill-sponsors')
        sponsors = sponsor_div.get_text(strip=True) if sponsor_div else None # need to program in some data cleaning - names are not seperating nicely.
        bill_sponsors.append(sponsors)

        
        print(f"Processed: {bill_number}")
        bill_data_df = pd.DataFrame({
        'Bill Number': bill_numbers,
        'Title': bill_titles,
        'Summary': bill_summaries,
        'Session': bill_sessions,
        'Sponsors': bill_sponsors
    })



In [4]:
#bill_data_df.to_csv("bill_data_df(Sponsors).csv")
bill_data_df = pd.read_csv('bill_data_df(Sponsors).csv')
bill_data_df
    

,Unnamed: 0,Bill Number,Title,Summary,Session,Sponsors
0,0,HB16-1001,State Contract Certify Compliance With Equal P...,The bill requires a business that bids for a c...,Session:2016 Regular Session,Prime SponsorsRepresentativeJessie DanielsonRe...
1,1,HB16-1002,Employee Leave Attend Child's Academic Activities,"In 2009, the general assembly enacted the 'Par...",Session:2016 Regular Session,Prime SponsorsRepresentativeJanet BucknerSenat...
2,2,HB16-1003,Middle Class College Savings Act,For purposes of calculating a taxpayer's Color...,Session:2016 Regular Session,Prime SponsorsRepresentativeBrittany Pettersen...
3,3,HB16-1004,Measurable Goals Deadlines Colorado Climate Ac...,The bill requires:The state climate action pla...,Session:2016 Regular Session,Prime SponsorsRepresentativeFaith WinterRepres...
4,4,HB16-1005,Residential Precipitation Collection,Section 1 of the act allows the collection of ...,Session:2016 Regular Session,Prime SponsorsRepresentativeDaneya EsgarRepres...
...,...,...,...,...,...,...
1937,1937,HB20-1318,Standards For Recording Plats,The act allows county clerk and recorders to r...,Session:2020 Regular Session,Prime SponsorsRepresentativeShannon BirdRepres...
1938,1938,HB20-1319,Prohibit Sale Of Flavored Nicotine Products,"On and after September 1, 2020, the bill prohi...",Session:2020 Regular Session,Prime SponsorsRepresentativeYadira CaraveoRepr...
1939,1939,HB20-1320,Off-highway Vehicle Certificates Of Title,Current law requires an off-highway vehicle to...,Session:2020 Regular Session,Prime SponsorsRepresentativeKevin Van WinkleRe...
1940,1940,HB20-1321,State Bid Preference For EPA Environmental Pro...,The SmartWay transport partnership program of ...,Session:2020 Regular Session,Prime SponsorsRepresentativeBri Buentello


### Importing CSV data from EXCEL

In [5]:
co_stats_df = pd.read_csv('20241101_statistics.csv')
gender_status_stats_df = pd.read_csv('20241101gender_status_statistics.csv')


In [6]:
co_stats_df

,Total Registered Voters by Party & Status \nas of 00:17 on 11/01/2024,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40
0,Active registered voters,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All,NaN
1,County,ACN,APV,CTR,DEM,FWD,GRN,LBR,NOL,REP,...,FWD,GRN,LBR,NOL,REP,UAF,UNI,Prereg Total,County,Grand Total
2,Adams,"1,308",809,236,"91,779",22,627,"2,897","2,236","59,911",...,0,10,9,78,330,"5,414",2,"6,110",Adams,"360,518"
3,Alamosa,60,24,2,"2,745",0,20,75,52,"2,731",...,0,0,0,1,11,168,0,185,Alamosa,"11,988"
4,Arapahoe,"1,248",701,300,"125,374",29,853,"3,829","2,297","84,239",...,2,10,12,118,520,"6,603",0,"7,839",Arapahoe,"487,712"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Washington,14,2,1,195,0,2,15,6,"2,134",...,0,0,0,0,22,26,0,48,Washington,"3,699"
64,Weld,941,360,141,"42,146",9,379,"2,269","1,327","71,752",...,0,2,6,56,557,"3,967",1,"4,775",Weld,"261,220"
65,Yuma,21,3,3,519,0,3,35,12,"2,974",...,0,0,0,0,36,120,0,157,Yuma,"6,494"
66,Total,"11,746","4,990","3,200","1,046,870",248,"8,741","37,756","20,925","936,583",...,2,68,131,"1,060","6,348","55,172",15,"67,054",Total,"4,622,650"


In [7]:
gender_status_stats_df

,Total Registered Voters by Status\nas of 00:17 on 11/01/2024,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29
0,Active Registered Voters,NaN,NaN,NaN,NaN,NaN,Inactive Registered Voters,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,County,Female,Male,Not Disclosed,X,Active Total,County,Female,Male,Not Disclosed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adams,"160,523","155,043","1,295","1,602","318,463",Adams,"17,161","18,127",26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alamosa,"5,258","4,917",17,52,"10,244",Alamosa,753,780,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arapahoe,"216,316","204,844","1,711","1,462","424,333",Arapahoe,"26,894","28,104",18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Washington,"1,586","1,722",3,6,"3,317",Washington,166,159,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Weld,"116,441","112,818",771,602,"230,632",Weld,"12,136","13,451",9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,Yuma,"2,960","2,857",11,13,"5,841",Yuma,240,250,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,Total,"2,028,014","1,958,250","15,746","16,907","4,018,917",Total,"252,087","273,039",276,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
